In [18]:
from bs4 import BeautifulSoup
import requests
import pandas as pd



### Plan

- write function to get soup from each page (i.e. 10 results)
    - for each article, extract: title, link, year, site, first author
    - NB if returns empty page, stop the for loop
- do this for 3 links: MRz, MRs, BC
    - exclude all articles that are not in BC (all papers citing THE paper but are not actually on BC)

In [60]:
def scrape_url(url):
    # get header: http://myhttpheader.com/
    headers = {'User-Agent':
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Safari/537.36'}

    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content,'lxml')
    
  
    df_list = parse_soup(soup)
    return df_list
    
def parse_soup(soup):
    df_list = []
    
    num_articles = len(soup.select('[data-lid]'))
    
    for item in soup.select('[data-lid]'):
        try:
            # extarct from HTML
            title = [item.select('h3')[0].get_text()]
            link = [item.select('a')[0]['href']]
            abstract = [item.select('.gs_rs')[0].get_text()]
            info = item.select('.gs_a')[0].get_text()
            
            # parse info
            info_split = info.split("-")
            journal_year = info_split[1].split(", ")
            first_author = [info_split[0].split(", ")[0]]
        
            info_list = [*info_split, *journal_year , *first_author]
            info_list = [x.strip(' ') for x in info_list]
            info_list = [x.replace('\xa0', "") for x in info_list]
            article = [*title, info_list[4], info_list[5], info_list[2], info_list[3],  *link]

            df_list = df_list.append(article)  ##### WHY IS THIS NOT WORKING!!!?
            print(df_list)
            
        except Exception as e:
            #raise e 
            print('==cant parse this soup')
            
          
    return(df_list)
            
            

In [61]:
url='https://scholar.google.com/scholar?start=20&q=mendelian+randomization&hl=en&as_sdt=2005&sciodt=0,5&cites=6613969377995102680&scipsc=1'
df_list = scrape_url(url)
df_list
#df = pd.DataFrame(df_list, columns=['title', 'year', 'first_author', 'source_site', 'source_name', 'link'])
#df

None
==cant parse this soup
==cant parse this soup
==cant parse this soup
==cant parse this soup
==cant parse this soup
==cant parse this soup
==cant parse this soup
==cant parse this soup
==cant parse this soup


In [59]:
df_list